<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C1_SparkSQL_SQLContext_HiveContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('Tested',datetime.now(pytz.timezone('Asia/Calcutta')))

Tested 2023-04-04 12:32:52.132672+05:30


# SPARK SQL
If you wish to run HIVE natively under Hadoop please see this notebook [Hadoop and Hive](https://github.com/prithwis/KKolab/blob/main/KK_B2_Hadoop_and_Hive.ipynb)

# Install Spark

## Manual Install Deprecated

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz     --- this gives errors
!wget -q https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz     # Using Older Version 

#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz -- this gives errors
!tar xf spark-3.0.3-bin-hadoop3.2.tgz



# -----------------------------------------------
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
# os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

!pip install -q findspark  # findspark is no more required
#!pip install -q pyspark
import findspark
findspark.init()
# -----------------------------------------------
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

## PIP Install Spark

In [2]:
#!pip3 -q install pyspark==3.0.3    # Problem with newer spark versions resolved, no need to install older version
!pip3 -q install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.9 MB/s eta 0:00:00


<SparkContext master=local[*] appName=pyspark-shell>

# Load Data

In [3]:
# Data files downloaded from author's G-Drive
#!gdown https://drive.google.com/uc?id=1JJH24ZZaiJrEKValD--UtyFcWl7UanwV  # 2% data
!wget -q 'https://raw.githubusercontent.com/Praxis-QR/BDSN/main/data/OLAP/eCommerce_02PC_2021.csv' # 2% data

#!gdown https://drive.google.com/uc?id=1g7mJ0v4fkERW0HWc1eq-SHs_jvQ0N2Oe  # 100% data
!wget -q 'https://raw.githubusercontent.com/Praxis-QR/BDSN/main/data/OLAP/eCommerce_Full_2021.csv' # 100% data 

!ls -al

total 44896
drwxr-xr-x 1 root root     4096 Apr  4 07:04 .
drwxr-xr-x 1 root root     4096 Apr  4 07:02 ..
drwxr-xr-x 4 root root     4096 Mar 31 13:35 .config
-rw-r--r-- 1 root root   916817 Apr  4 07:04 eCommerce_02PC_2021.csv
-rw-r--r-- 1 root root 45038728 Apr  4 07:04 eCommerce_Full_2021.csv
drwxr-xr-x 1 root root     4096 Mar 31 13:36 sample_data


In [4]:
#we remove the CRLF character from the end of the row if it exists
!sed 's/\r//' /content/eCommerce_Full_2021.csv > datafile.csv
#!sed 's/\r//' /content/eCommerce_02PC_2021.csv > datafile.csv


#sed -i -e "1d" datafile.csv               # remove the first line containing headers from the file

In [5]:
data_file = 'datafile.csv'
raw_data = sc.textFile(data_file)
print ("Data Size", raw_data.count())

Data Size 541910


#Spark Data Frame

In [6]:
# CSV to Spark Dataframe 
eCommerce_df = spark.read.csv(data_file,inferSchema=True, header=True)
eCommerce_df.printSchema()          # Spark is smart enough to differentiate between text and numbers
                                    # Presence of comma in string fields has been handled correctly

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
eCommerce_df.select('Quantity','UnitPrice','Country').groupby(eCommerce_df.Country).count().sort('count', ascending = False).show()

+---------------+------+
|        Country| count|
+---------------+------+
| United Kingdom|495478|
|        Germany|  9495|
|         France|  8557|
|           EIRE|  8196|
|          Spain|  2533|
|    Netherlands|  2371|
|        Belgium|  2069|
|    Switzerland|  2002|
|       Portugal|  1519|
|      Australia|  1259|
|         Norway|  1086|
|          Italy|   803|
|Channel Islands|   758|
|        Finland|   695|
|         Cyprus|   622|
|         Sweden|   462|
|    Unspecified|   446|
|        Austria|   401|
|        Denmark|   389|
|          Japan|   358|
+---------------+------+
only showing top 20 rows



#Spark SQL
What is the difference between SQLContext and HiveContext? See [here](https://intellipaat.com/community/7599/what-is-the-difference-between-apache-spark-sqlcontext-vs-hivecontext#:~:text=HiveContext%20is%20a%20super%20set,read%20data%20from%20Hive%20tables.&text=The%20more%20basic%20SQLContext%20provides,does%20not%20depend%20on%20Hive.), or [here](https://stackoverflow.com/questions/33666545/what-is-the-difference-between-apache-spark-sqlcontext-vs-hivecontext)

In [8]:
eCommerce_df.createOrReplaceTempView("t_eCommerce")                    # Create a Spark Table

In [11]:
spark.sql("SELECT Country, count(*) count FROM t_eCommerce group by Country order by count desc").show()

+---------------+------+
|        Country| count|
+---------------+------+
| United Kingdom|495478|
|        Germany|  9495|
|         France|  8557|
|           EIRE|  8196|
|          Spain|  2533|
|    Netherlands|  2371|
|        Belgium|  2069|
|    Switzerland|  2002|
|       Portugal|  1519|
|      Australia|  1259|
|         Norway|  1086|
|          Italy|   803|
|Channel Islands|   758|
|        Finland|   695|
|         Cyprus|   622|
|         Sweden|   462|
|    Unspecified|   446|
|        Austria|   401|
|        Denmark|   389|
|          Japan|   358|
+---------------+------+
only showing top 20 rows



In [15]:
#Countries = spark.sql("SELECT * FROM t_eCommerce limit 5")
Countries = spark.sql("SELECT * FROM t_eCommerce")
Countries.show(5, truncate=False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
+---------+---------+-----------------------------------

In [25]:
spark.sql("select distinct Country from t_eCommerce").show()

+------------------+
|           Country|
+------------------+
|            Sweden|
|         Singapore|
|           Germany|
|            France|
|            Greece|
|European Community|
|           Belgium|
|           Finland|
|             Malta|
|       Unspecified|
|             Italy|
|              EIRE|
|         Lithuania|
|            Norway|
|             Spain|
|           Denmark|
|         Hong Kong|
|           Iceland|
|            Israel|
|   Channel Islands|
+------------------+
only showing top 20 rows



In [27]:
spark.sql("select count(distinct Country) CountryCount from t_eCommerce").show()

+------------+
|CountryCount|
+------------+
|          38|
+------------+



In [29]:
#Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
spark.sql("SELECT sum(Quantity), sum(UnitPrice), Country from t_eCommerce group by Country order by sum(Quantity) desc").show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|      4263823|2245712.9239976574| United Kingdom|
|       200128| 6492.549999999972|    Netherlands|
|       142637|48447.190000000934|           EIRE|
|       117448| 37666.00000000061|        Germany|
|       110480| 43031.99000000053|         France|
|        83653| 4054.749999999998|      Australia|
|        35637|1806.8300000000006|         Sweden|
|        30325| 6813.689999999986|    Switzerland|
|        26824| 12633.44999999992|          Spain|
|        25218| 814.8600000000002|          Japan|
|        23152| 7540.129999999979|        Belgium|
|        19247| 6529.059999999987|         Norway|
|        16180|          13037.54|       Portugal|
|        10666| 3786.850000000005|        Finland|
|         9479|3738.5500000000015|Channel Islands|
|         8188|1266.9499999999991|        Denmark|
|         7999| 3879.3900000000

In [30]:
#Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
spark.sql("SELECT sum(Quantity) Quantity, round(sum(UnitPrice)) xSumUnitPrice, Country from t_eCommerce group by Country order by Quantity desc").show(40)

+--------+-------------+--------------------+
|Quantity|xSumUnitPrice|             Country|
+--------+-------------+--------------------+
| 4263823|    2245713.0|      United Kingdom|
|  200128|       6493.0|         Netherlands|
|  142637|      48447.0|                EIRE|
|  117448|      37666.0|             Germany|
|  110480|      43032.0|              France|
|   83653|       4055.0|           Australia|
|   35637|       1807.0|              Sweden|
|   30325|       6814.0|         Switzerland|
|   26824|      12633.0|               Spain|
|   25218|        815.0|               Japan|
|   23152|       7540.0|             Belgium|
|   19247|       6529.0|              Norway|
|   16180|      13038.0|            Portugal|
|   10666|       3787.0|             Finland|
|    9479|       3739.0|     Channel Islands|
|    8188|       1267.0|             Denmark|
|    7999|       3879.0|               Italy|
|    6317|       3920.0|              Cyprus|
|    5234|      25109.0|          

In [31]:
spark.sql("SELECT sum(Quantity) Quantity, round(sum(UnitPrice * Quantity)) Sales, Country from t_eCommerce group by Country order by Quantity desc").show(40)

+--------+---------+--------------------+
|Quantity|    Sales|             Country|
+--------+---------+--------------------+
| 4263823|8187791.0|      United Kingdom|
|  200128| 284662.0|         Netherlands|
|  142637| 263277.0|                EIRE|
|  117448| 221698.0|             Germany|
|  110480| 197404.0|              France|
|   83653| 137077.0|           Australia|
|   35637|  36596.0|              Sweden|
|   30325|  56385.0|         Switzerland|
|   26824|  54775.0|               Spain|
|   25218|  35341.0|               Japan|
|   23152|  40911.0|             Belgium|
|   19247|  35163.0|              Norway|
|   16180|  29367.0|            Portugal|
|   10666|  22327.0|             Finland|
|    9479|  20086.0|     Channel Islands|
|    8188|  18768.0|             Denmark|
|    7999|  16891.0|               Italy|
|    6317|  12946.0|              Cyprus|
|    5234|   9120.0|           Singapore|
|    4827|  10154.0|             Austria|
|    4769|  10117.0|           Hon

In [32]:
# Where description contains comma
spark.sql("SELECT InvoiceNo, Description  from t_eCommerce where Description like '%,%'").show(truncate = False)

+---------+----------------------------------+
|InvoiceNo|Description                       |
+---------+----------------------------------+
|536381   |AIRLINE LOUNGE,METAL SIGN         |
|536394   |FANCY FONT BIRTHDAY CARD,         |
|536520   |TRAY, BREAKFAST IN BED            |
|536520   |SWISS ROLL TOWEL, CHOCOLATE  SPOTS|
|536524   |SWISS ROLL TOWEL, CHOCOLATE  SPOTS|
|536531   |BIRTHDAY CARD, RETRO SPOT         |
|536531   |FANCY FONT BIRTHDAY CARD,         |
|536532   |FANCY FONT BIRTHDAY CARD,         |
|536544   |TRAY, BREAKFAST IN BED            |
|536544   |ACRYLIC JEWEL ICICLE, PINK        |
|536544   |TUMBLER, BAROQUE                  |
|536544   |TUMBLER, NEW ENGLAND              |
|536544   |METAL SIGN,CUPCAKE SINGLE HOOK    |
|536544   |METAL SIGN,CUPCAKE SINGLE HOOK    |
|536544   |SET 3 RETROSPOT TEA,COFFEE,SUGAR  |
|536544   |ELEPHANT, BIRTHDAY CARD,          |
|C536548  |HOOK, 1 HANGER ,MAGIC GARDEN      |
|536559   |FEATHER PEN,COAL BLACK            |
|536559   |FE

In [21]:
spark.sql("select count(*) from t_eCommerce where Description like '%,%'").show()

+--------+
|count(1)|
+--------+
|    4796|
+--------+



# Data Cleaning

## Remove comma from data of CSV file <br>
The data contains commas which may cause during the load process

In [33]:
#we remove the CRLF character from the end of the row if it exists
!sed 's/\r//' /content/eCommerce_Full_2021.csv > datafile.csv
#!sed 's/\r//g' /content/eCommerce_02PC_2021.csv > datafile.csv
# remove the first line containing headers from the file
!sed -i -e "1d" datafile.csv               

Tropical beach, coffee has commas 

In [34]:
#!grep 'TROPICAL BEACH' datafile.csv
#!grep 'COFFEE,' datafile.csv
#!head datafile.csv

commas getting removed here

In [37]:
import pandas as pd
#p_df = pd.read_csv('datafile.csv',header=None)   # throws errors because of non utf-encoding 
                                                  # https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
p_df = pd.read_csv('datafile.csv',encoding = "ISO-8859-1",header=None)
#p_df = pd.read_csv('datafile.csv',encoding = "latin",header=None)
#p_df = pd.read_csv('datafile.csv',encoding = "latin")
p_df2 = p_df.replace(',', ' ', regex=True)
p_df2.to_csv('datafile2.csv',index=False,sep=',',header=False)
p_df2.shape

(541909, 8)

In [ ]:
#Desc2 = p_df2['Description'].str.encode('ascii','ignore').str.decode('ascii')
#p_df2['Description2'] = Desc2
#p_df2.head(3)
#p_df2.drop("Description", axis=1, inplace=True)
#p_df2.head(3)

In [39]:
#!head datafile2.csv
#!grep 'TROPICAL BEACH' datafile2.csv
#!grep 'COFFEE' datafile2.csv

##SQL Context

In [40]:
#from pyspark.sql import SQLContext
from pyspark.sql import Row

#sqlContext = SQLContext(sc)

In [41]:
data_file = "datafile2.csv"
raw_data = sc.textFile(data_file)
csv_data = raw_data.map(lambda l: l.split(","))

In [42]:
csv_data.take(5)

[['536365',
  '85123A',
  'WHITE HANGING HEART T-LIGHT HOLDER',
  '6',
  '12/1/2010 8:26',
  '2.55',
  '17850.0',
  'United Kingdom'],
 ['536365',
  '71053',
  'WHITE METAL LANTERN',
  '6',
  '12/1/2010 8:26',
  '3.39',
  '17850.0',
  'United Kingdom'],
 ['536365',
  '84406B',
  'CREAM CUPID HEARTS COAT HANGER',
  '8',
  '12/1/2010 8:26',
  '2.75',
  '17850.0',
  'United Kingdom'],
 ['536365',
  '84029G',
  'KNITTED UNION FLAG HOT WATER BOTTLE',
  '6',
  '12/1/2010 8:26',
  '3.39',
  '17850.0',
  'United Kingdom'],
 ['536365',
  '84029E',
  'RED WOOLLY HOTTIE WHITE HEART.',
  '6',
  '12/1/2010 8:26',
  '3.39',
  '17850.0',
  'United Kingdom']]

In [43]:
row_data = csv_data.map(lambda p: Row(
    InvoiceNo=p[0], 
    StockCode=p[1],
    Description=p[2],
    Quantity=int(p[3]),
    InvoiceDate=p[4],
    UnitPrice=float(p[5]),
    CustomerID=p[6],
    Country=p[7]
    )
)

In [44]:
row_data.take(2)

[Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity=6, InvoiceDate='12/1/2010 8:26', UnitPrice=2.55, CustomerID='17850.0', Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='71053', Description='WHITE METAL LANTERN', Quantity=6, InvoiceDate='12/1/2010 8:26', UnitPrice=3.39, CustomerID='17850.0', Country='United Kingdom')]

In [45]:
#eCommerce2_df = sqlContext.createDataFrame(row_data)
#eCommerce2_df.registerTempTable("eCommerce")
# spark 2 onwards ...
eCommerce2_df = spark.createDataFrame(row_data)
eCommerce2_df.createOrReplaceTempView("eCommerce")

In [46]:
#Countries = sqlContext.sql("SELECT * FROM eCommerce limit 20")
Countries = spark.sql("SELECT * FROM eCommerce limit 5")
Countries.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+



In [47]:
#Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries = spark.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries.show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|      4263829|2245715.4739978826| United Kingdom|
|       200128| 6492.549999999974|    Netherlands|
|       142637| 48447.19000000096|           EIRE|
|       117448|  37666.0000000006|        Germany|
|       110480| 43031.99000000051|         France|
|        83653| 4054.749999999998|      Australia|
|        35637|           1806.83|         Sweden|
|        30325| 6813.689999999986|    Switzerland|
|        26824|12633.449999999923|          Spain|
|        25218| 814.8600000000001|          Japan|
|        23152| 7540.129999999981|        Belgium|
|        19247| 6529.059999999985|         Norway|
|        16180|          13037.54|       Portugal|
|        10666| 3786.850000000004|        Finland|
|         9479| 3738.550000000003|Channel Islands|
|         8188|1266.9499999999998|        Denmark|
|         7999| 3879.3900000000

### Why is there a difference from t_eCommerce Data?

In [48]:
#Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
spark.sql("SELECT sum(Quantity), sum(UnitPrice), Country from t_eCommerce group by Country order by sum(Quantity) desc").show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|      4263823|2245712.9239976574| United Kingdom|
|       200128| 6492.549999999972|    Netherlands|
|       142637|48447.190000000934|           EIRE|
|       117448| 37666.00000000061|        Germany|
|       110480| 43031.99000000053|         France|
|        83653| 4054.749999999998|      Australia|
|        35637|1806.8300000000006|         Sweden|
|        30325| 6813.689999999986|    Switzerland|
|        26824| 12633.44999999992|          Spain|
|        25218| 814.8600000000002|          Japan|
|        23152| 7540.129999999979|        Belgium|
|        19247| 6529.059999999987|         Norway|
|        16180|          13037.54|       Portugal|
|        10666| 3786.850000000005|        Finland|
|         9479|3738.5500000000015|Channel Islands|
|         8188|1266.9499999999991|        Denmark|
|         7999| 3879.3900000000

## HIVE context
this part adapted from [this notebook](https://colab.research.google.com/github/jmbanda/BigDataProgramming_2019/blob/master/Chapter_5_Loading_and_Saving_Data_in_Spark.ipynb#scrollTo=lOcCViaN4cwC)

In [49]:
from pyspark.sql import Row
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)


/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:718: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [50]:
data_file = "datafile2.csv"
raw_data = sc.textFile(data_file)
csv_data = raw_data.map(lambda l: l.split(","))

In [51]:
row_data = csv_data.map(lambda p: Row(
    InvoiceNo=p[0], 
    StockCode=p[1],
    Description=p[2],
    Quantity=int(p[3]),
    InvoiceDate=p[4],
    UnitPrice=float(p[5]),
    CustomerID=p[6],
    Country=p[7]
    )
)

In [52]:
ec_schema = spark.createDataFrame(row_data)
# Register it as a temp table
sqlContext.registerDataFrameAsTable(ec_schema, "ec_HiveTable")
sqlContext.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |ec_hivetable|       true|
|         |   ecommerce|       true|
|         | t_ecommerce|       true|
+---------+------------+-----------+



In [53]:
Countries = sqlContext.sql("SELECT * FROM ec_HiveTable limit 20")
Countries.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [57]:
Countries = sqlContext.sql("SELECT sum(Quantity) Quantity, round(sum(UnitPrice * Quantity)) Sales, Country from eCommerce group by Country order by Quantity desc")
Countries.show()

+--------+---------+---------------+
|Quantity|    Sales|        Country|
+--------+---------+---------------+
| 4263829|8187806.0| United Kingdom|
|  200128| 284662.0|    Netherlands|
|  142637| 263277.0|           EIRE|
|  117448| 221698.0|        Germany|
|  110480| 197404.0|         France|
|   83653| 137077.0|      Australia|
|   35637|  36596.0|         Sweden|
|   30325|  56385.0|    Switzerland|
|   26824|  54775.0|          Spain|
|   25218|  35341.0|          Japan|
|   23152|  40911.0|        Belgium|
|   19247|  35163.0|         Norway|
|   16180|  29367.0|       Portugal|
|   10666|  22327.0|        Finland|
|    9479|  20086.0|Channel Islands|
|    8188|  18768.0|        Denmark|
|    7999|  16891.0|          Italy|
|    6317|  12946.0|         Cyprus|
|    5234|   9120.0|      Singapore|
|    4827|  10154.0|        Austria|
+--------+---------+---------------+
only showing top 20 rows



In [ ]:
from datetime import datetime
import pytz
print('signed off at  ',datetime.now(pytz.timezone('Asia/Kolkata')))

signed off at   2022-11-22 07:47:55.797537+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)